<a href="https://colab.research.google.com/github/huggingface/blog/blob/notebook_update_may15/notebooks/01_how_to_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# We won't need TensorFlow here
#!pip uninstall -y tensorflowhttps://colab.research.google.com/github/huggingface/blog/blob/notebook_update_may15/notebooks/01_how_to_train.ipynb
# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-f5iryb41
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-f5iryb41
  Created wheel for transformers: filename=transformers-3.0.2-py3-none-any.whl size=810726 sha256=45d4934ad42be8cfdae98c361f328971061def6f3c9b8f8cf9b960c76d8fd661
  Stored in directory: /tmp/pip-ephem-wheel-cache-by11bkn9/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
tokenizers                         0.8.1rc2           
transformers                       3.0.2              


In [4]:
import pandas as pd
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /home/taylor/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
stop = stopwords.words('english')
temp = []
for element in stop:
    temp.append(" " + element + " ")
#     print(element)
# print(stop)
stop = temp
print(stop)

#custom
Remove= [' and ',' the ',' is ',' this ',' a ', '\n', '[', ']', "'", "\\n", '"']
Remove= Remove + stop

data = pd.read_csv('reddit.csv', error_bad_lines=False);
# We only need the Headlines text column from the data

# data_text['index'] = data_text.index
data_text = []
for key, value in data[['text']].T.to_dict().items():
    value['text'] = value['text'].lower()
    
    for rep in Remove:
        value['text']=value['text'].replace(rep, " ")
        
    if len(value['text']) <=512:
        data_text.append(value['text'])

# for element in data_text:
#     for thing in Remove:
#         element = element.replace(thing,"")



# data = data_text..tolist()
# data = data.replace("\n"," ")
# data = data.replace("["," ")
# 
# data = data.replace("]"," ")
# data = data.replace("]"," ")
# data = data.apply(lambda x: [item for item in x if item not in stop])
# data = data.apply(lambda x: [item for item in x if item not in Remove])
# data = data.replace(Remove,"")

# print (data)
print(data_text)





[' i ', ' me ', ' my ', ' myself ', ' we ', ' our ', ' ours ', ' ourselves ', ' you ', " you're ", " you've ", " you'll ", " you'd ", ' your ', ' yours ', ' yourself ', ' yourselves ', ' he ', ' him ', ' his ', ' himself ', ' she ', " she's ", ' her ', ' hers ', ' herself ', ' it ', " it's ", ' its ', ' itself ', ' they ', ' them ', ' their ', ' theirs ', ' themselves ', ' what ', ' which ', ' who ', ' whom ', ' this ', ' that ', " that'll ", ' these ', ' those ', ' am ', ' is ', ' are ', ' was ', ' were ', ' be ', ' been ', ' being ', ' have ', ' has ', ' had ', ' having ', ' do ', ' does ', ' did ', ' doing ', ' a ', ' an ', ' the ', ' and ', ' but ', ' if ', ' or ', ' because ', ' as ', ' until ', ' while ', ' of ', ' at ', ' by ', ' for ', ' with ', ' about ', ' against ', ' between ', ' into ', ' through ', ' during ', ' before ', ' after ', ' above ', ' below ', ' to ', ' from ', ' up ', ' down ', ' in ', ' out ', ' on ', ' off ', ' over ', ' under ', ' again ', ' further ', ' th

In [6]:
# print(stop)


In [7]:


# data['text']
# temp = [] 
# for key, value in data.T.to_dict().items():
# #     print(element)
#     if len(value['text']) <= 512:
#         temp.append(value)
    
# temp

with open("RedditText.txt", "w") as output:
    for element in data_text:
        output.write(element + "\n")


In [8]:
%%bash 
# ls
cat RedditText.txt

1
  reality tv gone far.  
  endorsed elon! ,  .. funny!  
   deleted   
  trump 100% told to sway black vote biden  
   distraction (https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/president-elect-donald-trump-and-kanye-west-stand-together-news-photo-1586966279.jpg?resize=768:*)  
  everyone laughing like everyone laughed trump first announced  
  what’s gambit? ,  water young black vote biden?  
  another loser wants trump publicity stunt. ,  prior album release.  
  literally going go way vote him.  
  hero need!  
  hate year.  
  kanye great he’s clown  
  shit. ,  imagine kanye president..  i’ll take trump.  
  fuck point. ,  also fuck kardashians.  
  well stole 40 year old white hipster vote biden.  
  entirely designed draw black vote young vote away biden help trump.  
  say voting traitor-in-chief. ,   kanye west signals vote donald trump 2020:  know voting  (https://www.businessinsider.com/kanye-west-signals-hell-vote-for-donald-trump-in-2020-2020-4)  
  cynica

In [9]:
%%time 
from pathlib import Path

from tokenizers import ByteLevelBPETokenizer

paths = [str(x) for x in Path(".").glob("**/*.txt")]

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

CPU times: user 2.11 s, sys: 613 ms, total: 2.72 s
Wall time: 141 ms


Now let's save files to disk

In [10]:
!rm -rf RedditText
!mkdir RedditText
tokenizer.save_model("RedditText")

['RedditText/vocab.json', 'RedditText/merges.txt']

🔥🔥 Wow, that was fast! ⚡️🔥

We now have both a `vocab.json`, which is a list of the most frequent tokens ranked by frequency, and a `merges.txt` list of merges.

Here’s  how you can use it in `tokenizers`, including handling the RoBERTa special tokens – of course, you’ll also be able to use it directly from `transformers`.


In [12]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

NameError: name 'BertProcessing' is not defined

In [13]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "./RedditText/vocab.json",
    "./RedditText/merges.txt",
)

## 3. Train a language model from scratch

**Update:** This section follows along the [`run_language_modeling.py`](https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_language_modeling.py) script, using our new [`Trainer`](https://github.com/huggingface/transformers/blob/master/src/transformers/trainer.py) directly. Feel free to pick the approach you like best.

> We’ll train a RoBERTa-like model, which is a BERT-like with a couple of changes (check the [documentation](https://huggingface.co/transformers/model_doc/roberta.html) for more details).

As the model is BERT-like, we’ll train it on a task of *Masked language modeling*, i.e. the predict how to fill arbitrary tokens that we randomly mask in the dataset. This is taken care of by the example script.


In [14]:
# Check that we have a GPU
!nvidia-smi

Sat Aug  1 23:12:29 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:06:00.0 Off |                    0 |
| N/A   67C    P0    60W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           Off  | 00000000:07:00.0 Off |                    0 |
| N/A   48C    P0    72W / 149W |      0MiB / 11441MiB |     99%      Default |
+-------------------------------+----------------------+----------------------+
        

In [15]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

### We'll define the following config for the model

In [16]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

Now let's re-create our tokenizer in transformers

In [17]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("./RedditText", max_len=512)

Finally let's initialize our model.

**Important:**

As we are training from scratch, we only initialize from a config, not from an existing pretrained model or checkpoint.

In [18]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [19]:
model.num_parameters()
# => 84 million parameters

84095008

### Now let's build our training Dataset

We'll build our dataset by applying our tokenizer to our text file.

Here, as we only have one text file, we don't even need to customize our `Dataset`. We'll just use the `LineByLineDataset` out-of-the-box.

In [20]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="RedditText.txt",
    block_size=64,
)

CPU times: user 576 ms, sys: 55.7 ms, total: 631 ms
Wall time: 26.2 ms


Like in the [`run_language_modeling.py`](https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_language_modeling.py) script, we need to define a data_collator.

This is just a small helper that will help us batch different samples of the dataset together into an object that PyTorch knows how to perform backprop on.

In [21]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

### Finally, we are all set to initialize our Trainer

In [22]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./EsperBERTo",
    overwrite_output_dir=True,
    num_train_epochs=40,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

### Start training

In [23]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


/home/taylor/anaconda3/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '




CPU times: user 3min 43s, sys: 2min 5s, total: 5min 48s
Wall time: 3min 10s


TrainOutput(global_step=160, training_loss=7.3459084063768385)

#### 🎉 Save final model (+ tokenizer + config) to disk

In [24]:
trainer.save_model("./EsperBERTo")

In [25]:
tokenizer.save_pretrained("./EsperBERTo")

('./EsperBERTo/vocab.json',
 './EsperBERTo/merges.txt',
 './EsperBERTo/special_tokens_map.json',
 './EsperBERTo/added_tokens.json')

## 4. Check that the LM actually trained

Aside from looking at the training and eval losses going down, the easiest way to check whether our language model is learning anything interesting is via the `FillMaskPipeline`.

Pipelines are simple wrappers around tokenizers and models, and the 'fill-mask' one will let you input a sequence containing a masked token (here, `<mask>`) and return a list of the most probable filled sequences, with their probabilities.



In [26]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./EsperBERTo",
    tokenizer="./EsperBERTo"
)

/home/taylor/anaconda3/lib/python3.7/site-packages/transformers/modeling_auto.py:805: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [27]:
# The sun <mask>.
# =>

fill_mask("I will vote for <mask>.")

[{'sequence': '<s>I will vote for.</s>',
  'score': 0.0688648372888565,
  'token': 225,
  'token_str': 'Ġ'},
 {'sequence': '<s>I will vote for..</s>',
  'score': 0.050017476081848145,
  'token': 18,
  'token_str': '.'},
 {'sequence': '<s>I will vote for,.</s>',
  'score': 0.04943490028381348,
  'token': 281,
  'token_str': 'Ġ,'},
 {'sequence': '<s>I will vote for,.</s>',
  'score': 0.02165216952562332,
  'token': 16,
  'token_str': ','},
 {'sequence': '<s>I will vote for .</s>',
  'score': 0.011918372474610806,
  'token': 271,
  'token_str': 'ĠĠ'}]

Ok, simple syntax/grammar works. Let’s try a slightly more interesting prompt:



In [29]:
fill_mask("<mask> West")

# This is the beginning of a beautiful <mask>.
# =>

[{'sequence': '<s>  West</s>',
  'score': 0.9562067985534668,
  'token': 225,
  'token_str': 'Ġ'},
 {'sequence': '<s>   West</s>',
  'score': 0.00018933345563709736,
  'token': 271,
  'token_str': 'ĠĠ'},
 {'sequence': '<s>. West</s>',
  'score': 0.00010488053521839902,
  'token': 18,
  'token_str': '.'},
 {'sequence': '<s>, West</s>',
  'score': 0.0001039215421769768,
  'token': 16,
  'token_str': ','},
 {'sequence': '<s>’ West</s>',
  'score': 7.487100810976699e-05,
  'token': 322,
  'token_str': 'âĢĻ'}]